In [18]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [6]:
#print torch version
print(torch.__version__)

1.4.0


In [59]:
#set hyperparameters
batchSize = 128
imageSize = 64


In [60]:
#load custom dataset
transform=transforms.Compose([transforms.Scale(imageSize),transforms.ToTensor()])
dataset = dset.ImageFolder(root = 'C:/Users/satya/Downloads/archive/img_align_celeba/', transform=transform)
len(dataset)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2)

In [45]:
#creating the transformations
transform=transforms.Compose([transforms.Scale(imageSize),transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [21]:
#loading the dataset:
dataset=dset.CIFAR10(root='./data',download=True,transform=transform)
#dataset=dset.MNIST(root='./data',download=True,transform=transform)
dataloader=torch.utils.data.DataLoader(dataset,batch_size=batchSize,shuffle=True, num_workers=2)
len(dataloader)

Files already downloaded and verified


391

In [50]:
#defining weights init function that takes an input a neural network m and will initialize all its weights
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [51]:
ngpu=1
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")


In [61]:
#defining the generator
class G(nn.Module):
    def __init__(self):
        super(G,self).__init__()
        self.main=nn.Sequential(
            nn.ConvTranspose2d(100,512,4,1,0,bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512,256,4,2,1,bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256,128,4,2,1,bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128,64,4,2,1,bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64,3,4,2,1,bias=False),
            nn.Tanh()
            )
    def forward(self,input):
        output=self.main(input)
        return output
#creating the generator
Generator=G()
Generator.apply(weights_init)
#defining the discriminator

class D(nn.Module):
    def __init__(self):
        super(D,self).__init__()
        self.main=nn.Sequential(
            nn.Conv2d(3,64,4,2,1,bias=False),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(64,128,4,2,1,bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(128,256,4,2,1,bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(256,512,4,2,1,bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(512,1,4,1,0,bias=False),
            nn.Sigmoid()
        )
        
    def forward(self,input):
        output=self.main(input)
        return output.view(-1) #flattening the ouput
#creating the discriminator
Discriminator=D()
Discriminator.apply(weights_init)


#Training the DCGANs
criterion=nn.BCELoss()
#criteron=torch.nn.BCEWithLogitsLoss

optmizerD=optim.Adam(Discriminator.parameters(),lr=0.0002,betas=(0.5,0.999))
optimizerG=optim.Adam(Generator.parameters(),lr=0.0002,betas=(0.5,0.999))

for epoch in range(25):
    for i, data in enumerate(dataloader,0):
        #1st step: updating the weights of the discriminator
        Discriminator.zero_grad() #initialize gradients to zero
        #training the discriminator with real images
        real,_=data
        input=Variable(real) #converting the real image into a torch variable
        target=Variable(torch.ones(input.size()[0])) #creating a tensor of ones with the size of the minibatch
        output=Discriminator(input) #feeding the real image into the discriminator network
        error_real=criterion(output,target)

        #training the discriminator with fake images
        noise=Variable(torch.randn(input.size()[0],100,1,1)) #making the noise tensor
        fake=Generator(noise) # output of the generator taking the noise to generate 
        target=Variable(torch.zeros(input.size()[0])) 
        output=Discriminator(fake.detach()) #feeding the fake image into the discriminator, but detaching the gradeints
        error_false=criterion(output,target)

        #backpropagation of the total error
        total_error=error_real+error_false #total error
        total_error.backward() #backpropagating the error
        optmizerD.step() #for weight updation

        #updating weights of the generator
        Generator.zero_grad() #initialize gradients of generator to zero
        target=Variable(torch.ones(input.size()[0])) #creating a tensor of ones with the size of the minibatch
        output=Discriminator(fake) #feeding the fake into discriminator
        error_G=criterion(output,target)
        error_G.backward() #backpropagating the gradients in the generator
        optimizerG.step() #updating the weights of the generator

        #printing losses and saving real and generated images
        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch,25,i,len(dataloader),total_error.data,error_G.data))
        #save images every 100 steps or iternations
        if i%100==0:
            vutils.save_image(real,'%s/real_samples.png' %"C:/Users/satya/Downloads/Azure/Azure_cp/samples/results1")
            fake=Generator(noise)
            vutils.save_image(fake.data,'%s/fake_samples_epoch_%03d.png'% ("C:/Users/satya/Downloads/Azure/Azure_cp/samples/results1",epoch))







RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 67108864 bytes. Buy new RAM!


G()

D(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): LeakyReLU(negative_slope=0.2, inplace=True)
    (12): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (13): Sigm

In [30]:
#Discriminator.parameters()
Generator.parameters()

<generator object Module.parameters at 0x000001EDE35ED228>

ValueError: optimizer got an empty parameter list

Files already downloaded and verified
[0/25][0/782] Loss_D: 1.9885 Loss_G: 6.5058
[0/25][1/782] Loss_D: 1.2612 Loss_G: 4.3858
[0/25][2/782] Loss_D: 1.3176 Loss_G: 5.8346
[0/25][3/782] Loss_D: 1.0160 Loss_G: 6.3122
[0/25][4/782] Loss_D: 0.8345 Loss_G: 6.1848


KeyboardInterrupt: 